In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
import keras
import tensorflow as tf
from keras import optimizers
import pandas as pd
import numpy as np
import os

import cx_Oracle
from sqlalchemy import types, create_engine



from sklearn.model_selection import train_test_split

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

C:\python\Anaconda3\envs\deeplearning\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
datapath = os.listdir("../static/data/")
datapath = [i for i in datapath if 'A' in i]
dataname = [i[:7] for i in datapath]
print(datapath)
print(dataname)

['A000100_유한양행.csv', 'A000640_동아쏘시오홀딩스.csv', 'A001630_종근당.csv', 'A006280_녹십자.csv', 'A008930_한미사이언스.csv', 'A009290_광동제약.csv', 'A069620_대웅제약.csv', 'A096760_JW홀딩스.csv', 'A128940_한미약품.csv', 'A249420_일동제약.csv']
['A000100', 'A000640', 'A001630', 'A006280', 'A008930', 'A009290', 'A069620', 'A096760', 'A128940', 'A249420']


In [57]:
def read_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute( query )
        names = [ x[0] for x in cursor.description]
        rows = cursor.fetchall()
        return pd.DataFrame( rows, columns=names)
    finally:
        if cursor is not None:
            cursor.close()

In [58]:
# Data preprocessing
connect = cx_Oracle.connect("STOCKDJANGO", "dhruddnjs", "localhost/orcl")
target_DataFrame = read_query(connect,"select * from %s" % dataname[0])
target_DataFrame = target_DataFrame.set_index("STOCK_INDEX")
target_DataFrame = target_DataFrame.sort_index()

target_DataFrame.head()




,STOCK_DATE,STOCK_TIME,STOCK_PRICE,STOCK_VOLUME
STOCK_INDEX,,,,
0,20180102,1001,219000,206
1,20180102,1002,219000,68
2,20180102,1003,220000,407
3,20180102,1004,219000,317
4,20180102,1005,217500,88


In [59]:
tmp_DataFrame = target_DataFrame[["STOCK_PRICE","STOCK_VOLUME"]]
tmp_DataFrame = tmp_DataFrame.drop([0])
tmp_DataFrame = tmp_DataFrame.reindex()
tmp_DataFrame.rename(columns={"STOCK_PRICE" : "1DAY_PRICE","STOCK_VOLUME":"1DAY_VOLUME"},inplace=True)
tmp_DataFrame = tmp_DataFrame.reset_index(drop = True)

df_merge = pd.merge(target_DataFrame, tmp_DataFrame, how="outer", left_index=True, right_index=True)

df_merge.head()

,STOCK_DATE,STOCK_TIME,STOCK_PRICE,STOCK_VOLUME,1DAY_PRICE,1DAY_VOLUME
0,20180102,1001,219000,206,219000.0,68.0
1,20180102,1002,219000,68,220000.0,407.0
2,20180102,1003,220000,407,219000.0,317.0
3,20180102,1004,219000,317,217500.0,88.0
4,20180102,1005,217500,88,217500.0,160.0


In [60]:
tmp_DataFrame = target_DataFrame[["STOCK_PRICE","STOCK_VOLUME"]]
tmp_DataFrame = tmp_DataFrame.drop([0,1])
tmp_DataFrame = tmp_DataFrame.reindex()
tmp_DataFrame.rename(columns={"STOCK_PRICE" : "2DAY_PRICE","STOCK_VOLUME":"2DAY_VOLUME"},inplace=True)
tmp_DataFrame = tmp_DataFrame.reset_index(drop = True)

df_merge = pd.merge(df_merge, tmp_DataFrame, how="outer", left_index=True, right_index=True)


In [61]:

df_merge.head()

,STOCK_DATE,STOCK_TIME,STOCK_PRICE,STOCK_VOLUME,1DAY_PRICE,1DAY_VOLUME,2DAY_PRICE,2DAY_VOLUME
0,20180102,1001,219000,206,219000.0,68.0,220000.0,407.0
1,20180102,1002,219000,68,220000.0,407.0,219000.0,317.0
2,20180102,1003,220000,407,219000.0,317.0,217500.0,88.0
3,20180102,1004,219000,317,217500.0,88.0,217500.0,160.0
4,20180102,1005,217500,88,217500.0,160.0,217000.0,88.0


In [62]:
tmp_DataFrame = target_DataFrame["STOCK_PRICE"]
tmp_DataFrame = tmp_DataFrame.drop([0,1,2])
tmp_DataFrame = tmp_DataFrame.to_frame()
tmp_DataFrame = tmp_DataFrame.reindex()

tmp_DataFrame.rename(columns={"STOCK_PRICE" : "TARGET_PRICE"},inplace=True)
tmp_DataFrame = tmp_DataFrame.reset_index(drop = True)

df_merge = pd.merge(df_merge, tmp_DataFrame, how="outer", left_index=True, right_index=True)

In [63]:
df_merge.tail()
df_merge = df_merge.drop([74197,74198,74199])


In [67]:
df_merge = df_merge.drop("STOCK_DATE",axis=1)
df_merge = df_merge.drop("STOCK_TIME",axis=1)

dataset = df_merge.values
df_merge.head()

,STOCK_PRICE,STOCK_VOLUME,1DAY_PRICE,1DAY_VOLUME,2DAY_PRICE,2DAY_VOLUME,TARGET_PRICE
0,219000,206,219000.0,68.0,220000.0,407.0,219000.0
1,219000,68,220000.0,407.0,219000.0,317.0,217500.0
2,220000,407,219000.0,317.0,217500.0,88.0,217500.0
3,219000,317,217500.0,88.0,217500.0,160.0,217000.0
4,217500,88,217500.0,160.0,217000.0,88.0,217000.0


In [68]:


 # trained and test
X = dataset[:, 0:6]
Y = dataset[:, 6:7]

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y,test_size=0.3)

# model layers
model = Sequential()
model.add(Dense(18, input_dim=6, activation='relu'))
model.add(Dense(72, activation='relu'))
model.add(Dense(36, activation='relu'))
model.add(Dense(54, activation='relu'))
model.add(Dense(27, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(Dense(1))

adam = optimizers.Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy','mean_squared_error'])

model.fit(X_train, Y_train, epochs=200, batch_size=40)

Y_prediction = model.predict(X_validation).flatten()

# model save 
model.save('%s.h5' % dataname[i])

Epoch 1/200
51937/51937 [==============================] - 11s 204us/step - loss: 2357783853.4853 - acc: 0.0000e+00 - mean_squared_error: 2357783853.4853
Epoch 2/200
51937/51937 [==============================] - 9s 169us/step - loss: 1377385559.4906 - acc: 0.0000e+00 - mean_squared_error: 1377385559.4906
Epoch 3/200
51937/51937 [==============================] - 9s 176us/step - loss: 1404606673.9652 - acc: 0.0000e+00 - mean_squared_error: 1404606673.9652
Epoch 4/200
51937/51937 [==============================] - 9s 166us/step - loss: 1379489737.0029 - acc: 0.0000e+00 - mean_squared_error: 1379489737.0029
Epoch 5/200
51937/51937 [==============================] - 8s 161us/step - loss: 1413989905.2911 - acc: 0.0000e+00 - mean_squared_error: 1413989905.2911
Epoch 6/200
51937/51937 [==============================] - 8s 161us/step - loss: 1387463450.4776 - acc: 0.0000e+00 - mean_squared_error: 1387463450.4776
Epoch 7/200
51937/51937 [==============================] - 8s 162us/step - loss: 

51937/51937 [==============================] - 8s 159us/step - loss: 1392101146.1067 - acc: 0.0000e+00 - mean_squared_error: 1392101146.1067
Epoch 55/200
51937/51937 [==============================] - 8s 159us/step - loss: 1365837235.5307 - acc: 0.0000e+00 - mean_squared_error: 1365837235.5307
Epoch 56/200
51937/51937 [==============================] - 8s 159us/step - loss: 1365038032.4692 - acc: 0.0000e+00 - mean_squared_error: 1365038032.4692
Epoch 57/200
51937/51937 [==============================] - 9s 167us/step - loss: 1397042323.1617 - acc: 0.0000e+00 - mean_squared_error: 1397042323.1617
Epoch 58/200
51937/51937 [==============================] - 8s 160us/step - loss: 1385032328.1009 - acc: 0.0000e+00 - mean_squared_error: 1385032328.1009
Epoch 59/200
51937/51937 [==============================] - 8s 160us/step - loss: 1369427274.5593 - acc: 0.0000e+00 - mean_squared_error: 1369427274.5593
Epoch 60/200
51937/51937 [==============================] - 8s 159us/step - loss: 1354354

KeyboardInterrupt: 

In [74]:
#Data preprocessing
connect = cx_Oracle.connect("STOCKDJANGO", "dhruddnjs", "localhost/orcl")
target_DataFrame = read_query(connect,"select * from %s" % dataname[0])
target_DataFrame = target_DataFrame.set_index("STOCK_INDEX")
target_DataFrame = target_DataFrame.sort_index()

target_DataFrame.head()



# data drop
target_DataFrame = target_DataFrame.drop("STOCK_DATE",axis=1)
target_DataFrame = target_DataFrame.drop("STOCK_TIME",axis=1)

# label
tem_df = target_DataFrame["STOCK_PRICE"]
tem_df = tem_df.to_frame()
tem_df = tem_df.drop([0])
tem_df.rename(columns={"STOCK_PRICE":"PREDICT_PRICE"},inplace=True)
tem_df = tem_df.reset_index(drop = True)

# target dataset
df_merge = pd.merge(target_DataFrame, tem_df, how="outer", left_index=True, right_index=True)

df_merge.tail()
df_merge.drop([74199])
dataset = df_merge.values


In [ ]:
#

# trained and test
X = dataset[:, 0:2]
Y = dataset[:, 2:3]

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y,test_size=0.3)

# model layers
model = Sequential()
model.add(Dense(4, input_dim=2, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(4, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(1))

adam = optimizers.Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy','mean_squared_error'])

model.fit(X_train, Y_train, epochs=200, batch_size=40)

Y_prediction = model.predict(X_validation).flatten()


for j in range(10):
    real_price = Y_validation[j]
    predicted_price = Y_prediction[j]
    print('Real Price: {}, Predicted Price: {}'.format(str(real_price),str(predicted_price)))
    

In [84]:

for j in range(10):
    real_price = Y_validation[j]
    predicted_price = Y_prediction[j]
    print('Real Price: {}, Predicted Price: {}'.format(str(real_price),str(predicted_price)))


Real Price: [215000.], Predicted Price: 150407.78
Real Price: [250000.], Predicted Price: 174502.19
Real Price: [212500.], Predicted Price: 148661.66
Real Price: [225500.], Predicted Price: 157743.3
Real Price: [218500.], Predicted Price: 153210.55
Real Price: [206500.], Predicted Price: 144470.67
Real Price: [233000.], Predicted Price: 162977.7
Real Price: [233500.], Predicted Price: 163336.55
Real Price: [231500.], Predicted Price: 162281.47
Real Price: [215000.], Predicted Price: 150756.08


In [29]:
def json_save(datapath,result):
    with open("%s%s.json" % (datapath,"todayStock") , "w", encoding="utf-8") as file:
        json.dump(result, file ,ensure_ascii=False, indent="\t")            
    return None

,현재가,거래량,미래가
74195,206500,83,206500.0
74196,206500,103,206000.0
74197,206000,523,206500.0
74198,206500,48,206500.0
74199,206500,2740,NaN
